# 1导数数据

In [1]:
import warnings
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from gensim import corpora, models
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import jieba
tqdm.pandas()

warnings.filterwarnings('ignore')
# 读取微信公众号文章
Wechat_article1 = pd.read_excel(
    '2018-2019茂名（含自媒体）.xlsx', sheet_name=4)  # 微信公众号文章

Wechat_article2 = pd.read_excel(
    '2020-2021茂名（含自媒体）.xlsx', sheet_name=4)  # 微信公众号文章

train_data = pd.concat([Wechat_article1, Wechat_article2], axis=0)  # 微信公众号文章
# 合并标题和正文
train_data['text'] = train_data['公众号标题']+'\n'+train_data['正文']

# 2删除特殊符号及文本分词

In [2]:

def clearTxt(line):
    if line != '':
        line = str(line).strip()
        #去除文本中的英文和数字
        line = re.sub("[a-zA-Z0-9]", "", line)
        #只保留中文、大小写字母
        reg = "[^0-9A-Za-z\u4e00-\u9fa5]"
        line = re.sub(reg, '', line)
        #分词
        segList = jieba.cut(line, cut_all=False)
        segSentence = ''
        for word in segList:
            if word != '\t':
                segSentence += word + " "
    return segSentence.strip()


text = train_data['text'].progress_apply(clearTxt)

  0%|                                                 | 0/6286 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache
Loading model cost 0.947 seconds.
Prefix dict has been built successfully.
100%|██████████████████████████████████████| 6286/6286 [01:03<00:00, 99.57it/s]


# 3去除停止词

In [3]:
import codecs
#读取停顿词列表
stopword_list = [k.strip() for k in open(
    'cn_stopwords.txt', encoding='utf8').readlines() if k.strip() != '']
def stopWord(line):
    sentence = delstopword(line, stopword_list)
    return sentence
#删除停用词
def delstopword(line, stopkey):
    wordList = line.split(' ')
    sentence = ''
    for word in wordList:
        word = word.strip()
        if word not in stopkey:
            if word != '\t':
                sentence += word + " "
    return sentence.strip()


train_data['分词后文本'] = text
text2 = list(train_data['分词后文本'].progress_apply(stopWord))

100%|█████████████████████████████████████| 6286/6286 [01:01<00:00, 102.61it/s]


# 3余弦相似度计算文本相关性

In [4]:
train_data

,文章ID,公众号标题,发布时间,正文,text,分词后文本
0,1001,2018，对自己好一点,2018-01-02 17:28,2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，请对自己好一点\n一辈子很...,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...,对 自己 好 一点 的 旅程 已经 结束 的 未来 拉开 了 帷幕 新 的 一年 里 请 对...
1,1002,春节机票预订有窍门,2018-01-02 17:28,距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚至一票难求。在这里分享...,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...,春节 机票 预订 有 窍门 距离 春节 还有 一个多月 的 时间 在线 旅游 网站 的 春节...
2,1003,冬日旅游知多D,2018-01-03 17:32,960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\n但是冬季的严寒气...,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...,冬日 旅游 知 多万平方公里 的 祖国 大地 四季 都 有 独特 美景 冬天 的 旅行 也 ...
3,1004,2018冬季暖心之旅,2018-01-03 17:32,长按二维码，关注我们\n中心联系人：林小姐13709649096\n刘小姐135000781...,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...,冬季 暖心 之 旅长 按 二维码 关注 我们 中心 联系人 林小姐 刘小姐 罗 小姐 陈小姐...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明,2018-01-05 16:57,,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...,关于 粤 号 大客车 排气管 喷火 事件 的 情况 说明
...,...,...,...,...,...,...
5660,7282,报名截止时间延期！茂名“小手拉大手·文明随手拍”活动拍拍你~,2021-10-31 15:28,由市创文巩卫办、市教育局联合主办的“小手拉大手·文明随手拍”活动举办至今已有2个多月，吸引了...,报名截止时间延期！茂名“小手拉大手·文明随手拍”活动拍拍你~\n由市创文巩卫办、市教育局联合...,报名 截止 时间 延期 茂名 小手 拉大 手 文明 随手 拍 活动 拍拍 你 由市 创文 巩...
5661,7283,党史百年天天读 ·10月31日,2021-10-31 15:28,重要论述\n\n1984年10月31日\n\n邓小平会见缅甸总统、缅甸国务委员会主席吴山友，...,党史百年天天读 ·10月31日\n重要论述\n\n1984年10月31日\n\n邓小平会见缅...,党史 百年 天天 读 月 日 重要 论述 年月日 邓小平 会见 缅甸 总统 缅甸 国务委员会...
5662,7284,赞！茂名这两个村入选全国示范！,2021-10-31 16:42,近日\n农业农村部官网公布了\n第二批全国乡村治理示范村镇名单\n\n\n其中\n高州市根子...,赞！茂名这两个村入选全国示范！\n近日\n农业农村部官网公布了\n第二批全国乡村治理示范村镇...,赞 茂名 这 两个 村 入选 全国 示范 近日 农业 农村部 官网 公布 了 第二批 全国 ...
5663,7285,报名截止时间延期！茂名“小手拉大手·文明随手拍”活动拍拍你~,2021-10-31 16:42,由市创文巩卫办、市教育局联合主办的“小手拉大手·文明随手拍”活动举办至今已有2个多月，吸引了...,报名截止时间延期！茂名“小手拉大手·文明随手拍”活动拍拍你~\n由市创文巩卫办、市教育局联合...,报名 截止 时间 延期 茂名 小手 拉大 手 文明 随手 拍 活动 拍拍 你 由市 创文 巩...


In [5]:
###余弦相似度
def cosine_similarity(seg1,seg2) -> float:
    
    word_list = list(set([word for word in seg1 + seg2]))#建立词库
    word_count_vec_1 = []
    word_count_vec_2 = []
    for word in word_list:
        word_count_vec_1.append(seg1.count(word))#文本1统计在词典里出现词的次数
        word_count_vec_2.append(seg2.count(word))#文本2统计在词典里出现词的次数

    vec_1 = np.array(word_count_vec_1)
    vec_2 = np.array(word_count_vec_2)
    #余弦公式

    num = vec_1.dot(vec_2.T)
    denom = np.linalg.norm(vec_1) * np.linalg.norm(vec_2)
    cos = num / denom
    sim = 0.5 + 0.5 * cos

    return sim

In [6]:
##导入相关主题词
s1 = '旅游、活动、节庆、特产、交通、酒店、景区、景点、文创、文化、乡村旅游、民宿、假日、假期、游客、采摘、赏花、春游、踏青、康养、公园、滨海游、度假、农家乐、剧本杀、旅行、徒步、工业旅游、线路、自驾游、团队游、攻略、游记、包车、玻璃栈道、游艇、高尔夫、温泉'.replace('、', ' ')
relation_word=list(s1.split(" "))

In [7]:
#计算相似度，以字典形式存放
result={}
for i in range(len(text2)):
    sim=cosine_similarity([word for word in text2[i].split(" ") if word !=""],relation_word)
    result[i]=sim

In [10]:
##上文字典中的相似度取出，以列表形式存放
num=[]
for i in range(len(text2)):
    num.append(result[i])

In [11]:
##以文本相似度算文章是否相关，这里选了0.5
sim_num = num
label = []
for i in enumerate(sim_num):
    if i[1]>0.5:
        label.append('相关')
    else:
        label.append('不相关')
result1 = pd.DataFrame(columns=['文章ID','分类标签'])

result1['文章ID'] = [str(i+1).zfill(4) for i in range(len(train_data))]
result1['分类标签'] =label
result1

,文章ID,分类标签
0,0001,相关
1,0002,相关
2,0003,相关
3,0004,不相关
4,0005,不相关
...,...,...
6281,6282,相关
6282,6283,相关
6283,6284,相关
6284,6285,相关


# 4主观统计相关与不相关个数看效果，适当调整上文阈值

In [12]:
sum(1 for i in result1["分类标签"] if str(i) =="相关")

4387

In [13]:
sum(1 for i in result1["分类标签"] if str(i) =="不相关")

1899

# 5最终确定后导出数据

In [25]:
result1.to_csv('result1.csv',index=False)